In [1]:
from box import ConfigBox

In [2]:
d = {"key":"value","key1":"value1"}

In [4]:
d = ConfigBox(d)

In [5]:
d.key

'value'

### Downlaod Data from google drive

In [6]:
import gdown

In [7]:
url = "https://drive.google.com/file/d/1gvXLFxxoqawVubViRigjSK0m8VPsJHWM/view?usp=sharing"

In [8]:
type(url)

str

In [12]:
fileId = url.split("/")[-2]
fileId

'1gvXLFxxoqawVubViRigjSK0m8VPsJHWM'

In [13]:
prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+fileId, "SentimentDataFinal.zip")

Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1gvXLFxxoqawVubViRigjSK0m8VPsJHWM
From (redirected): https://drive.google.com/uc?/export=download&id=1gvXLFxxoqawVubViRigjSK0m8VPsJHWM&confirm=t&uuid=105f37c5-b9e6-483c-b0aa-390a3643b5ba
To: c:\Users\akish\SentimentAnalysis\SentimentAPI\research\SentimentDataFinal.zip
100%|██████████| 74.5M/74.5M [00:27<00:00, 2.67MB/s]


'SentimentDataFinal.zip'

In [14]:
import os

In [3]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI\\research'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI'

In [18]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [19]:
from Sentiment.constants import *
from Sentiment.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast

c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [32]:
data = pd.read_excel('artifacts\data_transformation\sentimentDataset\TransformedFinalData.xlsx')

In [7]:
%pip install torch

In [33]:
%pip install transformers==4.6.0


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# import torch
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [34]:
data.drop('Unnamed: 0',axis=1,inplace=True)
data.dropna(how="any",inplace=True)

In [35]:
Review=list(data['Review'])
sentiment=list(data['sentiment'])

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Review, sentiment, test_size = 0.20, random_state = 0)

In [37]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

Downloading: 100%|██████████| 232k/232k [00:00<00:00, 13.2MB/s]
Downloading: 100%|██████████| 466k/466k [00:00<00:00, 1.13MB/s]
Downloading: 100%|██████████| 28.0/28.0 [00:00<00:00, 55.2kB/s]


In [38]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)

In [39]:
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [40]:
import tensorflow as tf 

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),y_test
))

In [41]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./results/logs',
    logging_steps=10
)

In [42]:
data.isnull().sum()

Review       0
sentiment    0
dtype: int64

In [45]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model = model,
    args = training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_projector', 'vocab_transform']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'classifier', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

[2024-01-05 15:31:18,943: WARNING: control_flow: The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).]
[2024-01-05 15:31:19,062: WARNING: control_flow: The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.]


TypeError: in user code:

    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 697, in distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 639, in apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 622, in training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 744, in run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    File "c:\Users\akish.pothuri\python\Sentiment

In [44]:
trainer.train()

[2024-01-05 15:29:53,074: WARNING: control_flow: The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).]
[2024-01-05 15:29:54,715: WARNING: ag_logging: AutoGraph could not transform <bound method Socket.send of <zmq.Socket(zmq.PUSH) at 0x1fe87f52400>> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert]
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause

TypeError: in user code:

    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 697, in distributed_training_steps  *
        self.args.strategy.run(self.apply_gradients, inputs)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 639, in apply_gradients  *
        gradients = self.training_step(features, labels, nb_instances_in_global_batch)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 622, in training_step  *
        per_example_loss, _ = self.run_model(features, labels, True)
    File "c:\Users\akish.pothuri\python\SentimentAnalysis\SentimentAPI\cancer\lib\site-packages\transformers\trainer_tf.py", line 744, in run_model  *
        outputs = self.model(features, labels=labels, training=training)[:2]
    File "c:\Users\akish.pothuri\python\Sentiment

In [1]:
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI\\research'

In [2]:
import os
os.chdir("../")
%pwd

'c:\\Users\\akish.pothuri\\python\\SentimentAnalysis\\SentimentAPI'

In [9]:
import os
from Sentiment import logger
import keras
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

from Sentiment.entity.config_entity import ModelEvaluationConfig

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def loadModel(self):
        '''
        Model Loading from the artifacts
        '''
        model = keras.models.load_model(os.path.join(self.config.model,"LSTMModel"+"V1"+".h5"))
        return model
    
    def predictTest(self,testSentences,model):
        '''
        test Sentence prediction
        '''
        predictions = model.predict(testSentences)
        return predictions
    
    def EvaluationMetrics(self,predictions,testLabels):
        '''
        Finding evaluation metrics
        '''
        try:
            accuracyScore = accuracy_score(testLabels,predictions)
            recallScore = recall_score(testLabels,predictions)
            precisionScore = precision_score(testLabels,predictions)
            f1Score = f1_score(testLabels,predictions)
            return accuracyScore,recallScore,precisionScore,f1Score
        except Exception as e:
            raise e
        
    def ModelEvaluationStatus(self,accuracyScore,recallScore,precisionScore,f1Score) -> bool:
        '''
        storing the metrics results in a text file
        '''
        try:
            Evaluation_status = None

            all_files = os.listdir(os.path.join("artifacts","model_evaluation"))

            for file in all_files:
                if file not in self.config.ALL_REQUIRED_FILES:
                    Evaluation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}")
                else:
                    Evaluation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Evaluation status: {Evaluation_status}"+",")
                        f.write(f"accuracyScore: {accuracyScore}"+",")
                        f.write(f"recallScore: {recallScore}"+",")
                        f.write(f"precisionScore: {precisionScore}"+",")
                        f.write(f"f1Score: {f1Score}"+",")

            return Evaluation_status
        
        except Exception as e:
            raise e

In [4]:
from Sentiment.config.configuration import ConfigurationManager
from Sentiment.components.model_trainer import ModelTrainer
from Sentiment.components.model_evaluation import ModelEvaluation
from Sentiment import logger

In [5]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer_config = ModelTrainer(config=model_trainer_config)
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
logger.info(f"data loading Started")
sentiData = model_trainer_config.loadData()
logger.info(f"data loading Completed")
logger.info(f"data Lowercase convertion Started")
sentiData = model_trainer_config.convertToLower(sentiData)
logger.info(f"data Lowercase convertion Completed")
logger.info(f"data Splitting Started")
trainSentences, testSentences, trainLabels, testLabels = model_trainer_config.splitData(sentiData)
logger.info(f"data Splitting Completed")

[2024-01-05 12:51:31,588: INFO: common: yaml file: {yaml_file}]
[2024-01-05 12:51:31,594: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-05 12:51:31,597: INFO: common: yaml file: {yaml_file}]
[2024-01-05 12:51:31,602: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-05 12:51:31,606: INFO: common: created directory at: artifacts]
[2024-01-05 12:51:31,608: INFO: common: created directory at: artifacts/model_trainer]
[2024-01-05 12:51:31,610: INFO: common: created directory at: artifacts/model_evaluation]
[2024-01-05 12:51:31,611: INFO: 1760456339: data loading Started]
[2024-01-05 12:51:49,594: INFO: 1760456339: data loading Completed]
[2024-01-05 12:51:49,595: INFO: 1760456339: data Lowercase convertion Started]
[2024-01-05 12:51:49,844: INFO: model_trainer: Data converted to lowercase]
[2024-01-05 12:51:49,844: INFO: 1760456339: data Lowercase convertion Completed]
[2024-01-05 12:51:49,844: INFO: 1760456339: data Splitting Started]
[2024-01-

In [19]:
logger.info(f"Model Loading")
model = model_evaluation_config.loadModel()
logger.info(f"Model Loaded")

[2024-01-05 13:00:20,625: INFO: 3100721655: Model Loading]
[2024-01-05 13:00:21,105: INFO: 3100721655: Model Loaded]


In [8]:
type(model)

keras.engine.sequential.Sequential

In [14]:
import os
import json
import gzip
import nltk
import pandas as pd
from urllib.request import urlopen
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import keras
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import tensorflow as tf
from tensorflow import keras as ks
import time

In [15]:
def Tokens(plaintext):
        text = []
        # print(plaintext)
        text.append(plaintext)
        # print(text)
        vocab_size = 3000 # choose based on statistics
        oov_tok = ''
        max_length = 200
        # text = ["This is good i have ever ate"]
        tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
        tokenizer.fit_on_texts(text)
        sequences = tokenizer.texts_to_sequences(text)
        # print("sequences",sequences)
        # pad the sequence
        padded = pad_sequences(sequences, padding='post', maxlen=max_length)
        # print(padded)
        return padded

In [30]:
result = Tokens('worst food i have ever eat and not good food')
model.predict(result)

1/1 [==============================] - 0s 37ms/step


array([[0.7405616]], dtype=float32)

In [11]:
logger.info(f"Predicting Test Data")
predictions = model_evaluation_config.predictTest(model,testSentences)
logger.info(f"Predicting Test Data Completed")

[2024-01-05 12:54:17,505: INFO: 1520779367: Predicting Test Data]


AttributeError: 'Series' object has no attribute 'predict'

In [ ]:
logger.info(f"EvaluationMetrics Started")
accuracyScore,recallScore,precisionScore,f1Score = model_evaluation_config.EvaluationMetrics(predictions,testLabels)
logger.info(f"EvaluationMetrics Completed")
model_evaluation_config.ModelEvaluationStatus(accuracyScore,recallScore,precisionScore,f1Score)
logger.info(f"EvaluationMetrics Saved")

In [13]:
import os
import json
import gzip
import nltk
import pandas as pd
from urllib.request import urlopen
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import pad_sequences
from sklearn.model_selection import train_test_split
import keras
import sklearn
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import tensorflow as tf
from tensorflow import keras as ks
import time

In [ ]:
def Tokens(plaintext):
        text = []
        # print(plaintext)
        text.append(plaintext)
        # print(text)
        vocab_size = 3000 # choose based on statistics
        oov_tok = ''
        max_length = 200
        # text = ["This is good i have ever ate"]
        tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
        tokenizer.fit_on_texts(text)
        sequences = tokenizer.texts_to_sequences(text)
        # print("sequences",sequences)
        # pad the sequence
        padded = pad_sequences(sequences, padding='post', maxlen=max_length)
        # print(padded)
        return padded

In [ ]:
# prepare tokenizer and model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [47]:
%pip install torch

  Using cached torch-2.1.2-cp38-cp38-win_amd64.whl.metadata (26 kB)
Using cached torch-2.1.2-cp38-cp38-win_amd64.whl (192.3 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split

In [51]:
class CustomDataset(Dataset):
  def __init__(self, texts, labels, tokenizer, max_len=512):
    self.texts = texts
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
    return len(self.texts)

  def __getitem__(self, idx):
    text = str(self.texts[idx])
    label = torch.tensor(self.labels[idx])

    encoding = self.tokenizer(text, truncation=True, padding="max_length",
                              max_length=self.max_len)

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label
    }

In [54]:
# prepare tokenizer and model
from transformers import AutoTokenizer, AutoModelForSequenceClassification

checkpoint = 'distilbert-base-uncased'
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
